We provided 3 .py and 3 .ipynb file. If you want to run the code, you can run it in Google Colab.
To do so, the Google Drive link which includes the dataset paths alongside with model check points is given below.
https://drive.google.com/drive/folders/1rsKAKgeTv3WeX5A_hko5w8j2H4RdNcfU?usp=sharing

IMPORTANT NOTE: YOU CANNOT RUN THIS CODE IN YOUR COMPUTER, OR EVEN IN YOUR OWN COLAB ENVIRONMENT. YOU NEED TO HAVE .pkl AND .h5 FILES TO PROCEED. FOR THE FINAL PROJECT OF EE443 COURSE, EVERYTHING YOU NEED IS AVAILABLE IN THE DRIVE LINK PROVIDED ABOVE. CONNECT IT IN YOUR COLAB ENVIRONMENT, AND YOU ARE READY TO GO.

This code is now set to be using the latest checkpoint, only for **TESTING PURPOSES**. In this way, the code will not train anything. If you want to see training results, turn the flag below **False**.


1-Links to other Google Colab environments

1.   GRU + Attention Model:
https://colab.research.google.com/drive/1Q4ueHEDtLcCVnZBcPsjfo1VWOcGua2cN?usp=sharing
2.   GRU + Attention Model with GloVE word embedding: 
https://colab.research.google.com/drive/1VmJmFsE5JW0Hno3oLbKnfH5fDqnsuMjW?usp=sharing
3.   LSTM Model: 
https://colab.research.google.com/drive/1_haPqb1kS1oVQfQSURitltkC5faJTmnn?usp=sharing





In [ ]:
##########IF YOU WANT TO TRAIN, SET THE FOLLOWING FLAG FALSE##############
flag = True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = '/content/drive/MyDrive/Dataset/test_data.zip'
!cp "{zip_path}" .
!unzip -q test_data.zip
!rm test_data.zip

In [ ]:
zip_path = '/content/drive/MyDrive/Dataset/train_data.zip'
!cp "{zip_path}" .
!unzip -q train_data.zip
!rm train_data.zip

In [ ]:
import gc
import os
import pickle
from tqdm.notebook import tqdm
import collections
import random
import time
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, EfficientNetB7, efficientnet
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow.keras.utils
import h5py
import numpy as np
import requests
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
test_img_path = np.array(listdir("/content/content/drive/MyDrive/Dataset/test"))
#train_img_path =np.array(listdir("train"))
#for deleting the non-existent images
a = None
for i in test_img_path:
    try:
        a = imread("/content/content/drive/MyDrive/Dataset/test/"+ i )
    except:
        os.remove("/content/content/drive/MyDrive/Dataset/test/"+ i)



In [ ]:
train_img_path = np.array(listdir("/content/content/drive/MyDrive/Dataset/train"))
#train_img_path =np.array(listdir("train"))

#for deleting the non-existent images
a = None
for i in train_img_path:
    try:
        a = imread("/content/content/drive/MyDrive/Dataset/train/"+ i )
    except:
        os.remove("/content/content/drive/MyDrive/Dataset/train/"+ i)
os.remove("/content/content/drive/MyDrive/Dataset/train/59999 (1).jpg")
os.remove("/content/content/drive/MyDrive/Dataset/train/82782 (1).jpg")


In [ ]:
def load_image_inception(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img_index = int(image_path.numpy().decode().split('/')[-1].split('.')[0])
    return img, img_index

def load_image_inceptionEval(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    img_index = int(image_path.split('/')[-1].split('.')[0])
    return img, img_index

def load_image_efNet(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (600, 600))
    img = tf.image.convert_image_dtype(img,dtype=tf.float32)
    img = efficientnet.preprocess_input(img)
    img_index = int(image_path.numpy().decode().split('\\')[1].split('.')[0])
    return img, img_index



In [ ]:
train = h5py.File('/content/drive/MyDrive/Dataset/eee443_project_dataset_train.h5', 'r')
test = h5py.File('/content/drive/MyDrive/Dataset/eee443_project_dataset_test.h5', 'r')
#Captions for training images
train_cap = train['train_cap']
train_cap = np.array(train_cap)
#The indices of training images
train_imid = train['train_imid']
train_imid = np.array(train_imid) -1
#Pretrained feature vector for training images !!!USING IT WILL HALVE THE GRADE!!!!
train_ims = train['train_ims']
train_ims = np.array(train_ims)
#Flickr URLs for training images
train_url = train['train_url']
train_url = np.array(train_url)
#dictionary for converting words to vocabulary indices
word_code = train['word_code']
word_code = np.array(word_code)
#Captions for testing images
test_caps = test['test_caps']
test_caps = np.array(test_caps)
#The indices of testing images
test_imid = test['test_imid']
test_imid = np.array(test_imid) 
#Pretrained feature vector for testing images !!!USING IT WILL HALVE THE GRADE!!!!
test_ims = test['test_ims']
test_ims = np.array(test_ims)
#Flickr URLs for testing images
test_url = test['test_url']
test_url = np.array(test_url)

#convert words to map indices 
word_code[0].tolist()
word_indices = word_code[0].tolist()
word_indices = np.array(word_indices, dtype='int')
#convert words to np array
words = np.array(word_code.dtype.descr)[:,0]

#retrieve all images to variables to use in further processes 
test_img_path = np.array(listdir("/content/content/drive/MyDrive/Dataset/test"))
train_img_path =np.array(listdir("/content/content/drive/MyDrive/Dataset/train"))




In [ ]:
#Function to map index to real world
def getCaps(id):
    captionMatrix = []
    for i in np.where(train_imid == id)[0]:
        captionMatrix.append(train_cap[i, :])
    return np.array(captionMatrix)
#Function to map index to real world for test data
def getCapsTest(id):
    captionMatrix = []
    for i in np.where(test_imid == id)[0]:
        captionMatrix.append(test_caps[i, :])
    return np.array(captionMatrix)

#Function to map index to string array
def readCaps(captionMatrix):
    str = ""
    str_arr = []
    for k in captionMatrix:
        str = ""
        for i in k:
          indices = np.where(word_indices == i)[0][0]
          word = words[indices]
          if i == k[-1]:
              str = str + word
          else:
              str = str + word + " "
        str_arr.append(str)
    return str_arr
#Image loader for generating pickle file for train dataset
#Pickle file increases the process speed of data in training
def imloader(dataset):
    batch_size=500
    feature_lst = None
    index_lst = None
    captions = None
    with open("train_data.pkl", "wb") as outfile:
      for data in tqdm(dataset.batch(batch_size)):
          image = data[0]
          index = data[1].numpy()
          feature = inceptionv3_model(image).numpy()

          for i in range(feature.shape[0]):
              feature_lst = (feature[i].squeeze().reshape((64,2048)))
              index_lst = (index[i])
              captions = (np.array(getCaps(index[i])))
              pickle.dump((feature_lst, index_lst, captions), outfile)
      outfile.close()
      return None

#Image loader for generating pickle file for validation dataset
#Pickle file increases the process speed of data in validation      
def imloader_val(dataset):
    batch_size=500
    feature_lst = None
    index_lst = None
    captions = None
    with open("test_data.pkl", "wb") as outfile:
      for data in tqdm(dataset.batch(batch_size)):

          image = data[0]
          index = data[1].numpy()
          feature = inceptionv3_model(image).numpy()
          for i in range(feature.shape[0]):
              feature_lst = (feature[i].squeeze().reshape((64,2048)))
              index_lst = (index[i])
              captions = (np.array(getCaps(index[i])))
              pickle.dump((feature_lst, index_lst, captions), outfile)

      outfile.close()
    return None

readCaps(getCapsTest(53))

In [ ]:
#Getting pretrained InceptionV3 model 
inception = InceptionV3(weights='imagenet')
inceptionv3_model = Model(inception.input, inception.layers[-3].output)

In [ ]:
#Data loading and train validation split 80%-20%
trainDataset = tf.data.Dataset.list_files("/content/content/drive/MyDrive/Dataset/train/*.jpg", shuffle=True).map(lambda x: tf.py_function(load_image_inception, [x], [tf.float32, tf.int32]))
trainData = trainDataset.take(int(len(train_img_path)*0.80))
validationData = trainDataset.skip(int(len(train_img_path)*0.80))

In [ ]:
if(not flag):
  #Batchwise pickle generation
  imloader(trainData)
  imloader_val(validationData)

In [ ]:
#Data yield operation definition for Dataset object of Tensorflow
def data_loader(filename):
   with open(filename, "rb") as f:
        while 1:
            try:

                yield pickle.load(f)
            except EOFError:
                break

trainDataset_inception = tf.data.Dataset.from_generator(data_loader, args=['/content/drive/MyDrive/Dataset/train_data.pkl'], output_types=(tf.float32,tf.int32, tf.int32))
valDataset_inception = tf.data.Dataset.from_generator(data_loader, args=['/content/drive/MyDrive/Dataset/test_data.pkl'], output_types=(tf.float32,tf.int32, tf.int32))

In [ ]:
train_set_size = int(len(train_img_path)*0.8) 
#initilize settings for training
sample_set_size, embedding_dim, neuron_count = 3000, 256, 512
num_steps  = train_set_size // sample_set_size
# Inceptionv3 gives vector in dimention (64,2048)
features_shape = 2048
attention_features_shape = 64

#implementation of attention mechanism
#inspired from https://www.tensorflow.org/tutorials/text/image_captioning
#implementation from Show, Attend and Tell: Neural Image Caption Generation with Visual Attention
# can be accessed here https://arxiv.org/pdf/1502.03044.pdf
class Attention(tf.keras.Model):
  def __init__(self, neuron_count):
    super(Attention, self).__init__()
    #there are 3 dense layers
    self.W2 = tf.keras.layers.Dense(neuron_count)
    self.W1 = tf.keras.layers.Dense(neuron_count)
    self.V = tf.keras.layers.Dense(1)
    self.neuron_count = neuron_count
  def call(self, features, hidden):

    #prepare dimention to initilize it as 1x256 values which will be given to embedding matrix as a coefficient   
    attention_pre_step = tf.nn.tanh(self.W1(features) + self.W2(tf.expand_dims(hidden, 1))) 
    
    #print(attention_pre_step.shape)
    #generate attention scores
    score = self.V(attention_pre_step)
    

    #calculate soft attention from given scores 
    attention_weights = tf.nn.softmax(score, axis=1)

    #distribution of soft attention for each vector 1x256
    vector = tf.reduce_sum(attention_weights * features, axis=1)
    #print(vector.shape)
    return vector, attention_weights
#implementation of encoder mechanism
#inspired from https://www.tensorflow.org/tutorials/text/image_captioning
class Encoder(tf.keras.Model):
    #decreasing output of inceptionv3 model to make it fit into embedding matrix
    def __init__(self, embedding_row):
        super(Encoder, self).__init__()
        #one fully connected layer is enough for understanding relation
        self.fc = tf.keras.layers.Dense(embedding_row)
    def call(self, x):
        x = self.fc(x)
        #use ReLU activation function
        x = tf.nn.relu(x)
        return x
#implementation of decoder mechanism
#inspired from https://www.tensorflow.org/tutorials/text/image_captioning
class Decoder(tf.keras.Model):
  def __init__(self, embedding_row, neuron_count, vocab_size):
    super(Decoder, self).__init__()
    self.attention = Attention(neuron_count)
    #create embedding layer as matrix of shape vocabulary size 1004 x 256
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_row)

    self.gru = tf.keras.layers.GRU(neuron_count, return_state=True,
                                   return_sequences=True, recurrent_initializer='glorot_uniform')
    
    #sequention 2 fully connected layers to better understand word, feature interaction
    self.fc1 = tf.keras.layers.Dense(neuron_count)
    self.fc2 = tf.keras.layers.Dense(vocab_size)
    self.neuron_count = neuron_count

  def call(self, x, features, hidden):
    #get result of attention weights and vectors from attention block
    vector,_ = self.attention(features, hidden)
    
    y = self.embedding(x)
    #print(vector.shape)
    #print(y.shape)
    #pass sentence input through word embedding and concat embedding output with attention vectors
    x = tf.concat([tf.expand_dims(vector,1), self.embedding(x)], axis=-1)

    #give input of concated features to GRU layer 
    out, state = self.gru(x)
    
    #Pass data through fully connected layers and set shape according to desired output (1x17)
    x = self.fc1(out)
    x = tf.reshape(x, (-1, x.shape[2]))
    x = self.fc2(x)
    return x, state

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.neuron_count))
#Create adam optimizer which performs well on similar tasks
optimizer = tf.keras.optimizers.Adam()
#Use Sparse Categorical Crossentropy as loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
encoder = Encoder(embedding_dim)
decoder = Decoder(embedding_dim, neuron_count, 1004)

#loss function calculation 
def loss_function(real, pred):
  loss_call = loss_object(real, pred)
  loss_result = loss_call * tf.cast(tf.math.logical_not(tf.math.equal(real, 0)), dtype=loss_call.dtype)
  average_loss = tf.reduce_mean(loss_result)
  return average_loss


In [ ]:
#!cp -av  '/content/checkpoints/caglar_full_train2' "/content/drive/MyDrive/Dataset/checkpoints/"

In [ ]:
checkpoint_path = "./checkpoints/attention_with_gru"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=50)

In [ ]:

start_epoch = 0
#restore the latest checkpoint if there is any
if ckpt_manager.latest_checkpoint:
  path_to_latest_checkpoint = ckpt_manager.latest_checkpoint
  last_index_of_checkpoint = path_to_latest_checkpoint.split('-')
  start_epoch = int(last_index_of_checkpoint[-1])
  ckpt.restore(ckpt_manager.latest_checkpoint)

if(flag):
  ckpt.restore("/content/drive/MyDrive/Dataset/checkpoints/attention_with_gru/ckpt-4")
  start_epoch = 7

loss_plot = []

@tf.function
def train_step(img_tensor, ground_truth):
  loss = 0
  caption_count = ground_truth.shape[0]
  caption_length = ground_truth.shape[1]
  temp_caption = decoder.reset_state(batch_size=caption_count)

  dec_input = tf.expand_dims([1] * caption_count, 1)

  with tf.GradientTape() as tape:
      #pass inceptionv3 outputs through encoder
      features = encoder(img_tensor)
      for i in range(1, caption_length):
          #get output of decoder 
          predictions, temp_caption = decoder(dec_input, features, temp_caption)
          loss += loss_function(ground_truth[:, i], predictions)
          #populate words to add into decoder input as next step
          dec_input = tf.expand_dims(ground_truth[:, i], 1)

  #declare conponents for backpropagation to calculate gradients
  trainable_variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients, trainable_variables))
  return loss, (loss / int(caption_length))

num_of_total_epochs = 7

for epoch in range(start_epoch, num_of_total_epochs):
    start = time.time()
    total_loss = 0
    avg_batch_loss_total = 0
    counter = 0
    #epoch += 1
    for (batch, (img_tensor, index, ground_truth)) in enumerate(trainDataset_inception):
        batch_loss, t_loss = train_step(img_tensor, ground_truth)
        total_loss = total_loss + t_loss
        counter += 1
        if batch % sample_set_size == 0:
            average_batch_loss = batch_loss.numpy()/int(ground_truth.shape[1])
            avg_batch_loss_total += average_batch_loss
            print('Epoch {} Batch {} Loss {}'.format(epoch+1,batch,average_batch_loss))

      
    loss_plot.append(avg_batch_loss_total / num_steps) 
    #save each epoch 
    ckpt_manager.save()

    print('Epoch {} Loss {}'.format(epoch+1, total_loss/num_steps ))
    print('Time taken for 1 epoch {} sec\n'.format(time.time()-start))

In [ ]:
plt.plot(loss_plot)
plt.show()

In [ ]:
@tf.function
def validation(img_tensor, target):
    loss = 0
    caption_count = ground_truth.shape[0]
    caption_length = ground_truth.shape[1]
    temp_caption = decoder.reset_state(batch_size=caption_count)
    dec_input = tf.expand_dims([1] * caption_count, 1)
    with tf.GradientTape() as tape:
        #pass inceptionv3 outputs through encoder
        features = encoder(img_tensor)
        for i in range(1, caption_length):
            #get output of decoder 
            predictions, temp_caption = decoder(dec_input, features, temp_caption)
            loss += loss_function(ground_truth[:, i], predictions)
            #populate words to add into decoder input as next step
            dec_input = tf.expand_dims(ground_truth[:, i], 1)
    return loss, (loss / int(caption_length))

In [ ]:
if(not flag):
  num_of_total_epochs = 7
  validation_loss = []
  for epoch in range(1, num_of_total_epochs):
      
      path = ckpt_manager.latest_checkpoint
      index = path.find("ckpt-")
      path = path[:index+5] + str(epoch) + path[index + 6:]
      ckpt.restore(path)
      start = time.time()
      total_loss = 0
      avg_batch_loss_total = 0
      counter = 0
      #epoch += 1
      for (batch, (img_tensor, index, targets)) in enumerate(valDataset_inception):
          batch_loss, t_loss = validation(img_tensor, targets)
          total_loss += t_loss
          counter += 1
          if batch % sample_set_size == 0:
              average_batch_loss = batch_loss.numpy()/int(targets.shape[1])
              avg_batch_loss_total += average_batch_loss
              print('Epoch {} Batch {} Loss {}'.format(epoch,batch,average_batch_loss))
      print(total_loss / counter)
      validation_loss.append(total_loss / counter)
      

In [ ]:
if(not flag):
  temp_loss = []
  for y, loss in enumerate(validation_loss):
    temp_loss.append(loss)
  plt.plot(loss_plot, label = 'Train Loss')
  plt.plot(temp_loss, label = 'Validation Loss')
  plt.xlabel("Epoch")
  plt.ylabel("Sparse Categorical Crossentropy")
  plt.title("Sparse Categorical Crossentropy vs Epoch for Validation and Train Data")
  plt.legend()
  plt.show()

In [ ]:
valgen = data_loader("/content/drive/MyDrive/Dataset/test_data.pkl")

In [ ]:
# for i in range(100):
#   iterated = next(valgen)

In [ ]:
def evaluate(image):
    #prepare input
    hidden = decoder.reset_state(batch_size=1)
    inception_out = inceptionv3_model(tf.expand_dims(load_image_inceptionEval(image)[0],0))
    inception_out = tf.reshape(inception_out, (inception_out.shape[0], -1, inception_out.shape[3]))
    #pass through encoder
    features = encoder(inception_out)
    temp_input = tf.expand_dims(tf.expand_dims(1, 0), axis=0)
    result = []
    #make prediction for all words iteratively
    for i in range(17):
        predictions, hidden = decoder(temp_input,features,hidden)
        pred_categoric_out = tf.random.categorical(predictions, 1)
        pred_categoric_out = pred_categoric_out[0][0]
        ids = pred_categoric_out.numpy()
        dec_output_word = tf.compat.as_text(words[np.where(word_indices == ids)][0])
        result.append(dec_output_word)
        if dec_output_word == 'x_END_':
            return result
        temp_input = tf.expand_dims([ids], 0)
    return result
#call eval function with given data
iterated = next(valgen)
image = "/content/content/drive/MyDrive/Dataset/train/"+ str(iterated[1])+".jpg"
result = evaluate(image)
res = imread(image)
capt_real = readCaps(getCaps(iterated[1]))
temp_cap = capt_real[0]
temp_cap = temp_cap[9:temp_cap.find(' x_END_')]
plt.title('Real Caption:'+temp_cap + '\n'+'Prediction Caption:' + ' '.join(result[:-1]))
plt.imshow(res)
plt.show()

ValueError: ignored

In [ ]:
def evaluate_test(image):
    hidden = decoder.reset_state(batch_size=1)
    #pass through encoder
    features = encoder(image)
    temp_input = tf.expand_dims(tf.expand_dims(1, 0), axis=0)
    result = []
    #make prediction for all words iteratively
    for i in range(17):
        predictions, hidden = decoder(temp_input,features,hidden)
        pred_categoric_out = tf.random.categorical(predictions, 1)
        pred_categoric_out = pred_categoric_out[0][0]
        ids = pred_categoric_out.numpy()
        dec_output_word = tf.compat.as_text(words[np.where(word_indices == ids)][0])
        result.append(dec_output_word)
        if dec_output_word == 'x_END_':
            return result
        temp_input = tf.expand_dims([ids], 0)
    return result

In [ ]:
!pip install pycocoevalcap

In [ ]:
def test_data_loader(dataset, batch_size):
    feature_lst = []
    index_lst = []
    captions = []
    for data in tqdm(dataset.batch(batch_size).take(100)):

        image = data[0]
        index = data[1].numpy()
        feature = inceptionv3_model(image).numpy()

        for i in range(feature.shape[0]):
            feature_lst.append(feature[i].squeeze().reshape((64,2048)))
            index_lst.append(index[i])
            captions.append(np.array(getCapsTest(index[i])))

    return feature_lst, index_lst, captions

test_img_path = np.array(listdir("/content/content/drive/MyDrive/Dataset/test"))

In [ ]:
testDataset = tf.data.Dataset.list_files("/content/content/drive/MyDrive/Dataset/test/*.jpg", shuffle=True).map(lambda x: tf.py_function(load_image_inception, [x], [tf.float32, tf.int32]))
testDataset_subset = testDataset.take(int(len(test_img_path)*0.01))

In [ ]:
features_test, index_test, caption_test = test_data_loader(testDataset_subset,100)

In [ ]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider 
from pycocoevalcap.meteor.meteor import Meteor

def score(ground_truth, prediction):
    

    score_dict = {}
    metrics = [(Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
                (Meteor(),"METEOR"),
                (Rouge(), "ROUGE_L"),
                (Cider(), "CIDEr")]
    for scorer, method in metrics:

        score, scores = scorer.compute_score(ground_truth, prediction)

        if type(score) == list:
            for m, s in zip(method, score):

                score_dict[m] = s

        else:  
            score_dict[method] = score

    b1,b2,b3,b4 = score_dict["Bleu_1"] * 100, score_dict["Bleu_2"] * 100, score_dict["Bleu_3"] * 100, score_dict["Bleu_4"] * 100
    m, r, c = score_dict["METEOR"] * 100, score_dict["ROUGE_L"] * 100,  score_dict["CIDEr"] * 100
    blue_scores = "BLEU-1: {} BLEU-2: {} BLEU-3: {} BLEU-4 {} ".format(b1,b2,b3,b4)
    other_scores = "METEOR:  {} ROGUE_L: {} CIDEr: {} ".format(m, r, c)
    print(blue_scores)
    print(other_scores)
    return score_dict
   

In [ ]:
import cv2
%matplotlib inline

In [ ]:
start = 0
indexes = index_test
blue_1 = 0
blue_2 = 0
blue_3 = 0
blue_4 = 0
meteor = 0
rouge_l = 0
cider = 0
counter = 0
for i, index in enumerate(indexes):
  predictions = {}
  gt_caption = {}
  captions = caption_test[i+start]
  testim = imread('/content/content/drive/MyDrive/Dataset/test/'+str(index)+'.jpg')
 
  for k in range(len(captions)):
    prediction_array_of_words = evaluate_test(features_test[i+start])
    pred_sentence = [' '.join(prediction_array_of_words[:-1])]
    predictions['0'] = pred_sentence
    capt_real = readCaps(getCapsTest(index))[k]
    capt_real = capt_real[9:capt_real.find(' x_END_')]
    gt_caption['0'] = [capt_real]
    print("################SEPERATOR###########")
    print(gt_caption)
    print(predictions)
    plt.title('Real Caption:'+ capt_real + '\n'+'Prediction Caption:' + pred_sentence[0])
    plt.imshow(testim)
    plt.show()  
    #print(temp)
    score_output = score(predictions, gt_caption)
    blue_1 +=  score_output['Bleu_1']
    blue_2 +=  score_output['Bleu_2']
    blue_3 +=  score_output['Bleu_3']
    blue_4 +=  score_output['Bleu_4']
    meteor +=  score_output['METEOR']
    rouge_l +=  score_output['ROUGE_L']
    cider +=  score_output['CIDEr']
    counter += 1

    #print(gt_caption[counter] )
blue_1 = blue_1 /  counter 
blue_2 = blue_2 /  counter 
blue_3 = blue_3 /  counter 
blue_4 = blue_4 /  counter 
meteor = meteor /  counter 
rouge_l = rouge_l / counter
cider = cider / counter
print(f"Blue 1 {blue_1}, Blue 2 {blue_2} ,Blue 3 {blue_3},Blue 4 {blue_4}, Meteor {meteor}, rouge_l {rouge_l}, cider {cider}" )